In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
%load_ext tensorboard
# CIFAR_10 is a set of 60K images 32x32 pixels on 3 channels
IMG_CHANNELS = 3
IMG_ROWS = 32
IMG_COLS = 32
#constant
BATCH_SIZE = 128
EPOCHS = 20
CLASSES = 10
VERBOSE = 1
VALIDATION_SPLIT = 0.2
OPTIM = tf.keras.optimizers.RMSprop()

2024-04-21 14:14:47.739137: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-21 14:14:47.739164: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-21 14:14:47.740086: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-21 14:14:47.745409: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-21 14:14:48.516162: W tensorflow/compiler/tf2

In [2]:
(X_train, y_train), (X_test, y_test) = datasets.cifar10.load_data()
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
 
#normalize 
mean = np.mean(X_train,axis=(0,1,2,3))
std = np.std(X_train,axis=(0,1,2,3))
X_train = (X_train-mean)/(std+1e-7)
X_test = (X_test-mean)/(std+1e-7)

y_train = tf.keras.utils.to_categorical(y_train, CLASSES)
y_test = tf.keras.utils.to_categorical(y_test, CLASSES)

170498071/170498071 [==============================] - 8s 0us/step


In [3]:
#image augmentation
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    )
datagen.fit(X_train)

In [4]:
def build(): 
    model = models.Sequential()
    
    #1st block
    model.add(layers.Conv2D(32, (3,3), padding='same', 
        input_shape=X_train.shape[1:], activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.2))
    #2nd block
    model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.3))
    #3d block 
    model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.4))
    #dense  
    model.add(layers.Flatten())
    model.add(layers.Dense(CLASSES, activation='softmax'))
    return model
    

In [5]:
callbacks = [tf.keras.callbacks.TensorBoard(log_dir='./logs/f')]

In [6]:
model = build()

In [7]:
model.compile(loss='categorical_crossentropy',optimizer=OPTIM,
              metrics=['accuracy'])

In [8]:
model.fit(X_train, y_train, batch_size=BATCH_SIZE,
          epochs=EPOCHS, validation_split=VALIDATION_SPLIT,
          verbose= VERBOSE,callbacks=callbacks)
score=model.evaluate(X_test,y_test,
                     batch_size=BATCH_SIZE,verbose= VERBOSE)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])

2024-04-21 14:15:02.591727: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 491520000 exceeds 10% of free system memory.
2024-04-21 14:15:02.933480: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 491520000 exceeds 10% of free system memory.


Epoch 1/20


2024-04-21 14:15:04.576049: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-04-21 14:15:04.733431: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-04-21 14:15:05.712316: I external/local_xla/xla/service/service.cc:168] XLA service 0x72cbccc00780 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-21 14:15:05.712333: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2024-04-21 14:15:05.717629: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1713701705.773314   23162 device_compiler.h:186] Compiled cluster using XL

313/313 [==============================] - ETA: 0s - loss: 1.9169 - accuracy: 0.4269

2024-04-21 14:15:09.608539: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 122880000 exceeds 10% of free system memory.
2024-04-21 14:15:09.700819: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 122880000 exceeds 10% of free system memory.


313/313 [==============================] - 7s 9ms/step - loss: 1.9169 - accuracy: 0.4269 - val_loss: 2.0587 - val_accuracy: 0.3920
Epoch 2/20
313/313 [==============================] - 2s 6ms/step - loss: 1.2606 - accuracy: 0.5936 - val_loss: 1.5266 - val_accuracy: 0.5890
Epoch 3/20
313/313 [==============================] - 2s 6ms/step - loss: 1.0023 - accuracy: 0.6694 - val_loss: 0.8941 - val_accuracy: 0.6923
Epoch 4/20
313/313 [==============================] - 2s 6ms/step - loss: 0.8772 - accuracy: 0.7125 - val_loss: 0.8745 - val_accuracy: 0.7080
Epoch 5/20
313/313 [==============================] - 2s 6ms/step - loss: 0.7903 - accuracy: 0.7383 - val_loss: 0.7138 - val_accuracy: 0.7555
Epoch 6/20
313/313 [==============================] - 2s 6ms/step - loss: 0.7079 - accuracy: 0.7630 - val_loss: 0.6846 - val_accuracy: 0.7687
Epoch 7/20
313/313 [==============================] - 2s 6ms/step - loss: 0.6370 - accuracy: 0.7842 - val_loss: 0.6644 - val_accuracy: 0.7799
Epoch 8/20
313/31

2024-04-21 14:15:44.243941: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 122880000 exceeds 10% of free system memory.
2024-04-21 14:15:54.318934: W external/local_tsl/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 390.6KiB (rounded to 400128)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-04-21 14:15:54.318957: I external/local_tsl/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2024-04-21 14:15:54.318964: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 159, Chunks in use: 158. 39.8KiB allocated for chunks. 39.5KiB in use in bin. 6.6KiB client-requested in use in bin.
2024-04-21 14:15:54.318971: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	T

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [ ]:
%tensorboard --logdir logs/f

In [ ]:
model_json = model.to_json()
with open('cifar10_architecture.json', 'w') as json_file:
    json_file.write(model_json)

In [ ]:
model.save_weights('model.h5')

In [ ]:
scores = model.evaluate(X_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0])) 